## ML.NET Regression
Creates a regression model to predict the quality of wine using 11 physicochemical features

### NuGet package installation

In [23]:
#r "nuget:Microsoft.ML, 1.4.0"
#r "nuget:XPlot.Plotly, 3.0.1"

Installing package Microsoft.ML, version 1.4.0..done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package XPlot.Plotly, version 3.0.1..done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

### Namespaces

In [24]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using XPlot.Plotly;

### Input Class Definition

In [25]:
public class RegressionData
{
    [LoadColumn(0)]
    public float FixedAcidity;

    [LoadColumn(1)]
    public float VolatileAcidity;

    [LoadColumn(2)]
    public float CitricAcid;

    [LoadColumn(3)]
    public float ResidualSugar;

    [LoadColumn(4)]
    public float Chlorides;

    [LoadColumn(5)]
    public float FreeSulfurDioxide;

    [LoadColumn(6)]
    public float TotalSulfurDioxide;

    [LoadColumn(7)]
    public float Density;

    [LoadColumn(8)]
    public float Ph;

    [LoadColumn(9)]
    public float Sulphates;

    [LoadColumn(10)]
    public float Alcohol;

    [LoadColumn(11)]
    public float Label;
}


### Output Class Definition

In [26]:
public class RegressionPrediction
{
    [ColumnName("Label")]
    public float Rating;

    [ColumnName("Score")]
    public float Confidence;
}

### Read the Raw Data

In [27]:
var mlContext = new MLContext(seed: null);

var trainingData = mlContext.Data.LoadFromTextFile<RegressionData>(
    "./WineQuality_White_Train.csv", 
    separatorChar: ';',
    hasHeader: true);

### Prepare the Data

In [28]:
var pipeline =
    mlContext.Transforms.ReplaceMissingValues(
        outputColumnName: "PreparedFixedAcidity",
        inputColumnName: "FixedAcidity",
        replacementMode: MissingValueReplacingEstimator.ReplacementMode.Mean)
    .Append(mlContext.Transforms.DropColumns("FixedAcidity"))
    .Append(mlContext.Transforms.Concatenate("Features",
        new[]
        {
          "PreparedFixedAcidity",
          "VolatileAcidity",
          "CitricAcid",
          "ResidualSugar",
          "Chlorides",
          "FreeSulfurDioxide",
          "TotalSulfurDioxide",
          "Density",
          "Ph",
          "Sulphates",
          "Alcohol"
        }))
    .Append(mlContext.Regression.Trainers.Sdca());

### Train the Model

In [29]:
var model = pipeline.Fit(trainingData);

### Evaluate the Model

In [30]:
// Load the raw test data.
var testData = mlContext.Data.LoadFromTextFile<RegressionData>(
    "./WineQuality_White_Test.csv", 
    separatorChar: ';',
    hasHeader: true);

// Score the test data and calculate the metrics.
var scoredData = model.Transform(testData);
var qualityMetrics =  mlContext.Regression.Evaluate(scoredData);
display(qualityMetrics);

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.5407283658524416,0.5099886612314429,0.7141349040842654,0.5099886618378868,0.15208854126913252


### Visualize the Quality Metrics

In [31]:
string[] metricNames = 
    { 
        "Mean Absolute Error", 
        "Mean Squared Error", 
        "Root Mean Squared Error", 
        "Loss Function", 
        "R Squared"
    };

double[] metricValues = 
    { 
        qualityMetrics.MeanAbsoluteError, 
        qualityMetrics.MeanSquaredError, 
        qualityMetrics.RootMeanSquaredError, 
        qualityMetrics.LossFunction, 
        qualityMetrics.RSquared
    };

var graph = new Graph.Bar()
{
    x = metricValues,
    y = metricNames,
    orientation = "h",
    marker = new Graph.Marker { color = "darkred" }
};

var chart = Chart.Plot(graph);

var layout = new Layout.Layout(){ title="Quality Metrics" };
chart.WithLayout(layout);

display(chart);

### TODO: create and use a prediction engine